# 지능 시스템 설계 중간 대체 프로젝트

### MNIST 데이터 셋에 대해서 hidden layer를 늘리면서 성능 비교하기

##### 수업에서 사용한 코드를 사용하기 때문에, 기존의 코딩 스타일과 다를 수 있으며, 해당 코드에서는 MNIST 데이터셋을 csv파일에서 읽어와서 사용하며, tensorflow는 쓰지 않고 파이썬 루프를 이용해서 학습시켰습니다.

In [1]:
# python notebook for Make Your Own Neural Network
# code for a 3-layer neural network, and code for learning the MNIST dataset
# (c) Tariq Rashid, 2016
# license is GPLv2

In [1]:
import numpy
# scipy.special for the sigmoid function expit()
import scipy.special
# library for plotting arrays
import matplotlib.pyplot
# ensure the plots are inside this notebook, not an external window
%matplotlib inline

In [2]:
# neural network class definition
class neuralNetwork:
    
    
    # initialise the neural network
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        # set number of nodes in each input, hidden, output layer
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        
        # link weight matrices, wih and who
        # weights inside the arrays are w_i_j, where link is from node i to node j in the next layer
        # w11 w21
        # w12 w22 etc 
        self.wih = numpy.random.normal(0.0, pow(self.inodes, -0.5), (self.hnodes, self.inodes))
        self.who = numpy.random.normal(0.0, pow(self.hnodes, -0.5), (self.onodes, self.hnodes))

        # learning rate
        self.lr = learningrate
        
        # activation function is the sigmoid function
        self.activation_function = lambda x: scipy.special.expit(x)
        
        pass

    
    # train the neural network
    def train(self, inputs_list, targets_list):
        # convert inputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T
        
        # calculate signals into hidden layer
        hidden_inputs = numpy.dot(self.wih, inputs)
        # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # calculate signals into final output layer
        final_inputs = numpy.dot(self.who, hidden_outputs)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
         # output layer error is the (target - actual)
        output_errors = targets - final_outputs
        # hidden layer error is the output_errors, split by weights, recombined at hidden nodes
        hidden_errors = numpy.dot(self.who.T, output_errors) 
        
        # update the weights for the links between the hidden and output layers
        self.who += self.lr * numpy.dot((output_errors * final_outputs * (1.0 - final_outputs)), numpy.transpose(hidden_outputs))
        
        # update the weights for the links between the input and hidden layers
        self.wih += self.lr * numpy.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), numpy.transpose(inputs))
        
        pass

    
    # query the neural network
    def query(self, inputs_list):
        # convert inputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        
        # calculate signals into hidden layer
        hidden_inputs = numpy.dot(self.wih, inputs)
        # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # calculate signals into final output layer
        final_inputs = numpy.dot(self.who, hidden_outputs)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs

In [3]:
# number of input, hidden and output nodes
input_nodes = 784
hidden_nodes = 200
output_nodes = 10

# learning rate
learning_rate = 0.1

# create instance of neural network
n = neuralNetwork(input_nodes,hidden_nodes,output_nodes, learning_rate)

In [5]:
# load the mnist training data CSV file into a list
training_data_file = open("mnist_train.csv", 'r')
training_data_list = training_data_file.readlines()
training_data_file.close()

# load the mnist test data CSV file into a list
test_data_file = open("mnist_test.csv", 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()

In [6]:
# train the neural network

# epochs is the number of times the training data set is used for training
epochs = 30

for e in range(epochs):
    # go through all records in the training data set
    for record in training_data_list:
        # split the record by the ',' commas
        all_values = record.split(',')
        # scale and shift the inputs
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        # create the target output values (all 0.01, except the desired label which is 0.99)
        targets = numpy.zeros(output_nodes) + 0.01
        # all_values[0] is the target label for this record
        targets[int(all_values[0])] = 0.99
        n.train(inputs, targets)
        pass
    # scorecard for how well the network performs, initially empty
    scorecard = []

    # go through all the records in the test data set
    for record in test_data_list:
        # split the record by the ',' commas
        all_values = record.split(',')
        # correct answer is first value
        correct_label = int(all_values[0])
        # scale and shift the inputs
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        # query the network
        outputs = n.query(inputs)
        # the index of the highest value corresponds to the label
        label = numpy.argmax(outputs)
        # append correct or incorrect to list
        if (label == correct_label):
            # network's answer matches correct answer, add 1 to scorecard
            scorecard.append(1)
        else:
            # network's answer doesn't match correct answer, add 0 to scorecard
            scorecard.append(0)
            pass

        pass

    # calculate the performance score, the fraction of correct answers
    scorecard_array = numpy.asarray(scorecard)
    print ("performance for ",1+e," epochs = ", scorecard_array.sum() / scorecard_array.size)
    pass

performance for  1  epochs =  0.9564
performance for  2  epochs =  0.9667
performance for  3  epochs =  0.9709
performance for  4  epochs =  0.9718
performance for  5  epochs =  0.9723
performance for  6  epochs =  0.9733
performance for  7  epochs =  0.9731
performance for  8  epochs =  0.9733
performance for  9  epochs =  0.9734
performance for  10  epochs =  0.9722
performance for  11  epochs =  0.9725
performance for  12  epochs =  0.9734
performance for  13  epochs =  0.9727
performance for  14  epochs =  0.9713
performance for  15  epochs =  0.9719
performance for  16  epochs =  0.9724
performance for  17  epochs =  0.9727
performance for  18  epochs =  0.9726
performance for  19  epochs =  0.972
performance for  20  epochs =  0.9704
performance for  21  epochs =  0.9704
performance for  22  epochs =  0.9693
performance for  23  epochs =  0.9693
performance for  24  epochs =  0.97
performance for  25  epochs =  0.9699
performance for  26  epochs =  0.9699
performance for  27  epo

### learning rate를 0.01로 해서 다시 해봄

In [15]:
# number of input, hidden and output nodes
input_nodes = 784
hidden_nodes = 200
output_nodes = 10

# learning rate
learning_rate = 0.01

# create instance of neural network
n_1 = neuralNetwork(input_nodes,hidden_nodes,output_nodes, learning_rate)

epochs = 15

for e in range(epochs):
    # go through all records in the training data set
    for record in training_data_list:
        # split the record by the ',' commas
        all_values = record.split(',')
        # scale and shift the inputs
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        # create the target output values (all 0.01, except the desired label which is 0.99)
        targets = numpy.zeros(output_nodes) + 0.01
        # all_values[0] is the target label for this record
        targets[int(all_values[0])] = 0.99
        n_1.train(inputs, targets)
        pass
    # scorecard for how well the network performs, initially empty
    scorecard = []

    # go through all the records in the test data set
    for record in test_data_list:
        # split the record by the ',' commas
        all_values = record.split(',')
        # correct answer is first value
        correct_label = int(all_values[0])
        # scale and shift the inputs
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        # query the network
        outputs = n_1.query(inputs)
        # the index of the highest value corresponds to the label
        label = numpy.argmax(outputs)
        # append correct or incorrect to list
        if (label == correct_label):
            # network's answer matches correct answer, add 1 to scorecard
            scorecard.append(1)
        else:
            # network's answer doesn't match correct answer, add 0 to scorecard
            scorecard.append(0)
            pass

        pass

    # calculate the performance score, the fraction of correct answers
    scorecard_array = numpy.asarray(scorecard)
    print ("performance for ",1+e," epochs = ", scorecard_array.sum() / scorecard_array.size)
    pass

performance for  1  epochs =  0.9075
performance for  2  epochs =  0.9284
performance for  3  epochs =  0.9398
performance for  4  epochs =  0.9483
performance for  5  epochs =  0.9539
performance for  6  epochs =  0.9587
performance for  7  epochs =  0.9614
performance for  8  epochs =  0.9635
performance for  9  epochs =  0.9657
performance for  10  epochs =  0.9673
performance for  11  epochs =  0.9699
performance for  12  epochs =  0.9714
performance for  13  epochs =  0.9723
performance for  14  epochs =  0.9732
performance for  15  epochs =  0.9743


### 추가로 몇 번 더 돌려봄

In [17]:
epochs = 5

for e in range(epochs):
    # go through all records in the training data set
    for record in training_data_list:
        # split the record by the ',' commas
        all_values = record.split(',')
        # scale and shift the inputs
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        # create the target output values (all 0.01, except the desired label which is 0.99)
        targets = numpy.zeros(output_nodes) + 0.01
        # all_values[0] is the target label for this record
        targets[int(all_values[0])] = 0.99
        n_1.train(inputs, targets)
        pass
    # scorecard for how well the network performs, initially empty
    scorecard = []

    # go through all the records in the test data set
    for record in test_data_list:
        # split the record by the ',' commas
        all_values = record.split(',')
        # correct answer is first value
        correct_label = int(all_values[0])
        # scale and shift the inputs
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        # query the network
        outputs = n_1.query(inputs)
        # the index of the highest value corresponds to the label
        label = numpy.argmax(outputs)
        # append correct or incorrect to list
        if (label == correct_label):
            # network's answer matches correct answer, add 1 to scorecard
            scorecard.append(1)
        else:
            # network's answer doesn't match correct answer, add 0 to scorecard
            scorecard.append(0)
            pass

        pass

    # calculate the performance score, the fraction of correct answers
    scorecard_array = numpy.asarray(scorecard)
    print ("performance for ",16+e," epochs = ", scorecard_array.sum() / scorecard_array.size)
    pass

performance for  16  epochs =  0.9752
performance for  17  epochs =  0.9758
performance for  18  epochs =  0.9764
performance for  19  epochs =  0.9765
performance for  20  epochs =  0.9768


In [18]:
epochs = 5

for e in range(epochs):
    # go through all records in the training data set
    for record in training_data_list:
        # split the record by the ',' commas
        all_values = record.split(',')
        # scale and shift the inputs
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        # create the target output values (all 0.01, except the desired label which is 0.99)
        targets = numpy.zeros(output_nodes) + 0.01
        # all_values[0] is the target label for this record
        targets[int(all_values[0])] = 0.99
        n_1.train(inputs, targets)
        pass
    # scorecard for how well the network performs, initially empty
    scorecard = []

    # go through all the records in the test data set
    for record in test_data_list:
        # split the record by the ',' commas
        all_values = record.split(',')
        # correct answer is first value
        correct_label = int(all_values[0])
        # scale and shift the inputs
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        # query the network
        outputs = n_1.query(inputs)
        # the index of the highest value corresponds to the label
        label = numpy.argmax(outputs)
        # append correct or incorrect to list
        if (label == correct_label):
            # network's answer matches correct answer, add 1 to scorecard
            scorecard.append(1)
        else:
            # network's answer doesn't match correct answer, add 0 to scorecard
            scorecard.append(0)
            pass

        pass

    # calculate the performance score, the fraction of correct answers
    scorecard_array = numpy.asarray(scorecard)
    print ("performance for ",21+e," epochs = ", scorecard_array.sum() / scorecard_array.size)
    pass

performance for  21  epochs =  0.977
performance for  22  epochs =  0.9773
performance for  23  epochs =  0.9775
performance for  24  epochs =  0.978
performance for  25  epochs =  0.9783


In [20]:
epochs = 5

for e in range(epochs):
    # go through all records in the training data set
    for record in training_data_list:
        # split the record by the ',' commas
        all_values = record.split(',')
        # scale and shift the inputs
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        # create the target output values (all 0.01, except the desired label which is 0.99)
        targets = numpy.zeros(output_nodes) + 0.01
        # all_values[0] is the target label for this record
        targets[int(all_values[0])] = 0.99
        n_1.train(inputs, targets)
        pass
    # scorecard for how well the network performs, initially empty
    scorecard = []

    # go through all the records in the test data set
    for record in test_data_list:
        # split the record by the ',' commas
        all_values = record.split(',')
        # correct answer is first value
        correct_label = int(all_values[0])
        # scale and shift the inputs
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        # query the network
        outputs = n_1.query(inputs)
        # the index of the highest value corresponds to the label
        label = numpy.argmax(outputs)
        # append correct or incorrect to list
        if (label == correct_label):
            # network's answer matches correct answer, add 1 to scorecard
            scorecard.append(1)
        else:
            # network's answer doesn't match correct answer, add 0 to scorecard
            scorecard.append(0)
            pass

        pass

    # calculate the performance score, the fraction of correct answers
    scorecard_array = numpy.asarray(scorecard)
    print ("performance for ",26+e," epochs = ", scorecard_array.sum() / scorecard_array.size)
    pass

performance for  26  epochs =  0.9785
performance for  27  epochs =  0.9784
performance for  28  epochs =  0.9784
performance for  29  epochs =  0.9784
performance for  30  epochs =  0.9781


# 4Layer에서의 구현

In [7]:
# neural network class definition
class neuralNetwork_4Layers:
    
    
     # initialise the neural network
    def __init__(self, inputnodes, hiddennodes1, hiddennodes2, outputnodes, learningrate):
        # set number of nodes in each input, hidden, output layer
        self.inodes = inputnodes
        self.hnodes1 = hiddennodes1
        self.hnodes2 = hiddennodes2
        self.onodes = outputnodes
        
        # link weight matrices, wih and who
        # weights inside the arrays are w_i_j, where link is from node i to node j in the next layer
        # w11 w21
        # w12 w22 etc 
        self.wih1 = numpy.random.normal(0.0, pow(self.inodes, -0.5), (self.hnodes1, self.inodes))
        self.wh1h2 = numpy.random.normal(0.0, pow(self.hnodes1, -0.5), (self.hnodes2, self.hnodes1))
        self.wh2o = numpy.random.normal(0.0, pow(self.hnodes2, -0.5), (self.onodes, self.hnodes2))

        # learning rate
        self.lr = learningrate
        
        # activation function is the sigmoid function
        self.activation_function = lambda x: scipy.special.expit(x)
        
        pass

    
    # train the neural network
    def train(self, inputs_list, targets_list):
        # convert inputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T
        
        # calculate signals into hidden layer
        hidden1_inputs = numpy.dot(self.wih1, inputs)
        # calculate the signals emerging from hidden layer
        hidden1_outputs = self.activation_function(hidden1_inputs)
        
        # calculate signals into hidden layer
        hidden2_inputs = numpy.dot(self.wh1h2, hidden1_outputs)
        # calculate the signals emerging from hidden layer
        hidden2_outputs = self.activation_function(hidden2_inputs)
        
        # calculate signals into final output layer
        final_inputs = numpy.dot(self.wh2o, hidden2_outputs)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
         # output layer error is the (target - actual)
        output_errors = targets - final_outputs
        # hidden layer error is the output_errors, split by weights, recombined at hidden nodes
        hidden2_errors = numpy.dot(self.wh2o.T, output_errors)
        hidden1_errors = numpy.dot(self.wh1h2.T, hidden2_errors)
        
        # update the weights for the links between the hidden and output layers
        self.wh2o += self.lr * numpy.dot((output_errors * final_outputs * (1.0 - final_outputs)), numpy.transpose(hidden2_outputs))
        self.wh1h2 += self.lr * numpy.dot((hidden2_errors * hidden2_outputs * (1.0 - hidden2_outputs)), numpy.transpose(hidden1_outputs))
        
        # update the weights for the links between the input and hidden layers
        self.wih1 += self.lr * numpy.dot((hidden1_errors * hidden1_outputs * (1.0 - hidden1_outputs)), numpy.transpose(inputs))
        
        pass

    
    # query the neural network
    def query(self, inputs_list):
        # convert inputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        
        # calculate signals into hidden layer
        hidden1_inputs = numpy.dot(self.wih1, inputs)
        # calculate the signals emerging from hidden layer
        hidden1_outputs = self.activation_function(hidden1_inputs)
        
        # calculate signals into hidden layer
        hidden2_inputs = numpy.dot(self.wh1h2, hidden1_outputs)
        # calculate the signals emerging from hidden layer
        hidden2_outputs = self.activation_function(hidden2_inputs)
        
        # calculate signals into final output layer
        final_inputs = numpy.dot(self.wh2o, hidden2_outputs)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs

In [8]:
# number of input, hidden and output nodes
input_nodes = 784
hidden_nodes1 = 500
hidden_nodes2 = 200
output_nodes = 10

# learning rate
learning_rate = 0.1

# create instance of neural network
n1 = neuralNetwork_4Layers(input_nodes,hidden_nodes1,hidden_nodes2, output_nodes, learning_rate)

In [3]:
# load the mnist training data CSV file into a list
training_data_file = open("mnist_train.csv", 'r')
training_data_list = training_data_file.readlines()
training_data_file.close()

In [4]:
# load the mnist test data CSV file into a list
test_data_file = open("mnist_test.csv", 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()

### epoch수를 하나씩 추가하면서 실행

In [12]:
epochs = 15

for e in range(epochs):
    # go through all records in the training data set
    for record in training_data_list:
        # split the record by the ',' commas
        all_values = record.split(',')
        # scale and shift the inputs
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        # create the target output values (all 0.01, except the desired label which is 0.99)
        targets = numpy.zeros(output_nodes) + 0.01
        # all_values[0] is the target label for this record
        targets[int(all_values[0])] = 0.99
        n1.train(inputs, targets)
        pass
    
    # scorecard for how well the network performs, initially empty
    scorecard = []

    # go through all the records in the test data set
    for record in test_data_list:
        # split the record by the ',' commas
        all_values = record.split(',')
        # correct answer is first value
        correct_label = int(all_values[0])
        # scale and shift the inputs
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        # query the network
        outputs = n1.query(inputs)
        # the index of the highest value corresponds to the label
        label = numpy.argmax(outputs)
        # append correct or incorrect to list
        if (label == correct_label):
            # network's answer matches correct answer, add 1 to scorecard
            scorecard.append(1)
        else:
            # network's answer doesn't match correct answer, add 0 to scorecard
            scorecard.append(0)
            pass

        pass

    # calculate the performance score, the fraction of correct answers
    scorecard_array = numpy.asarray(scorecard)
    print ("performance for ",1+e," epochs = ", scorecard_array.sum() / scorecard_array.size)
    pass

performance for  1  epochs =  0.9499
performance for  2  epochs =  0.9363
performance for  3  epochs =  0.9369
performance for  4  epochs =  0.9393
performance for  5  epochs =  0.9334
performance for  6  epochs =  0.921
performance for  7  epochs =  0.9071
performance for  8  epochs =  0.904
performance for  9  epochs =  0.918
performance for  10  epochs =  0.9098
performance for  11  epochs =  0.8768
performance for  12  epochs =  0.9118
performance for  13  epochs =  0.9116
performance for  14  epochs =  0.8939
performance for  15  epochs =  0.9077


### learning rate를 0.01로 해서 다시

In [9]:
n1_1 = neuralNetwork_4Layers(input_nodes,hidden_nodes1,hidden_nodes2, output_nodes, 0.01)

epochs = 30

for e in range(epochs):
    # go through all records in the training data set
    for record in training_data_list:
        # split the record by the ',' commas
        all_values = record.split(',')
        # scale and shift the inputs
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        # create the target output values (all 0.01, except the desired label which is 0.99)
        targets = numpy.zeros(output_nodes) + 0.01
        # all_values[0] is the target label for this record
        targets[int(all_values[0])] = 0.99
        n1_1.train(inputs, targets)
        pass
    
    # scorecard for how well the network performs, initially empty
    scorecard = []

    # go through all the records in the test data set
    for record in test_data_list:
        # split the record by the ',' commas
        all_values = record.split(',')
        # correct answer is first value
        correct_label = int(all_values[0])
        # scale and shift the inputs
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        # query the network
        outputs = n1_1.query(inputs)
        # the index of the highest value corresponds to the label
        label = numpy.argmax(outputs)
        # append correct or incorrect to list
        if (label == correct_label):
            # network's answer matches correct answer, add 1 to scorecard
            scorecard.append(1)
        else:
            # network's answer doesn't match correct answer, add 0 to scorecard
            scorecard.append(0)
            pass

        pass

    # calculate the performance score, the fraction of correct answers
    scorecard_array = numpy.asarray(scorecard)
    print ("performance for ",1+e," epochs = ", scorecard_array.sum() / scorecard_array.size)
    pass

performance for  1  epochs =  0.9227
performance for  2  epochs =  0.9482
performance for  3  epochs =  0.9588
performance for  4  epochs =  0.9668
performance for  5  epochs =  0.9703
performance for  6  epochs =  0.9728
performance for  7  epochs =  0.9752
performance for  8  epochs =  0.9759
performance for  9  epochs =  0.9773
performance for  10  epochs =  0.9776
performance for  11  epochs =  0.9779
performance for  12  epochs =  0.978
performance for  13  epochs =  0.9769
performance for  14  epochs =  0.9787
performance for  15  epochs =  0.9797
performance for  16  epochs =  0.9798
performance for  17  epochs =  0.9801
performance for  18  epochs =  0.9799
performance for  19  epochs =  0.9791
performance for  20  epochs =  0.979
performance for  21  epochs =  0.9794
performance for  22  epochs =  0.9795
performance for  23  epochs =  0.9784
performance for  24  epochs =  0.9805
performance for  25  epochs =  0.9798
performance for  26  epochs =  0.9792
performance for  27  ep

# 5layer에서 구현

In [5]:
# neural network class definition
class neuralNetwork_5Layers:
    
    
     # initialise the neural network
    def __init__(self, inputnodes, hiddennodes1, hiddennodes2, hiddennodes3, outputnodes, learningrate):
        # set number of nodes in each input, hidden, output layer
        self.inodes = inputnodes
        self.hnodes1 = hiddennodes1
        self.hnodes2 = hiddennodes2
        self.hnodes3 = hiddennodes3
        self.onodes = outputnodes
        
        # link weight matrices, wih and who
        # weights inside the arrays are w_i_j, where link is from node i to node j in the next layer
        # w11 w21
        # w12 w22 etc 
        self.wih1 = numpy.random.normal(0.0, pow(self.inodes, -0.5), (self.hnodes1, self.inodes))
        self.wh1h2 = numpy.random.normal(0.0, pow(self.hnodes1, -0.5), (self.hnodes2, self.hnodes1))
        self.wh2h3 = numpy.random.normal(0.0, pow(self.hnodes2, -0.5), (self.hnodes3, self.hnodes2))
        self.wh3o = numpy.random.normal(0.0, pow(self.hnodes3, -0.5), (self.onodes, self.hnodes3))

        # learning rate
        self.lr = learningrate
        
        # activation function is the sigmoid function
        self.activation_function = lambda x: scipy.special.expit(x)
        
        pass

    
    # train the neural network
    def train(self, inputs_list, targets_list):
        # convert inputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T
        
        # calculate signals into hidden layer
        hidden1_inputs = numpy.dot(self.wih1, inputs)
        # calculate the signals emerging from hidden layer
        hidden1_outputs = self.activation_function(hidden1_inputs)
        
        # calculate signals into hidden layer
        hidden2_inputs = numpy.dot(self.wh1h2, hidden1_outputs)
        # calculate the signals emerging from hidden layer
        hidden2_outputs = self.activation_function(hidden2_inputs)
        
        hidden3_inputs = numpy.dot(self.wh2h3, hidden2_outputs)
        hidden3_outputs = self.activation_function(hidden3_inputs)
        
        # calculate signals into final output layer
        final_inputs = numpy.dot(self.wh3o, hidden3_outputs)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
         # output layer error is the (target - actual)
        output_errors = targets - final_outputs
        # hidden layer error is the output_errors, split by weights, recombined at hidden nodes
        hidden3_errors = numpy.dot(self.wh3o.T, output_errors)
        hidden2_errors = numpy.dot(self.wh2h3.T, hidden3_errors)
        hidden1_errors = numpy.dot(self.wh1h2.T, hidden2_errors)
        
        # update the weights for the links between the hidden and output layers
        self.wh3o += self.lr * numpy.dot((output_errors * final_outputs * (1.0 - final_outputs)), numpy.transpose(hidden3_outputs))
        self.wh2h3 += self.lr * numpy.dot((hidden3_errors * hidden3_outputs * (1.0 - hidden3_outputs)), numpy.transpose(hidden2_outputs))
        self.wh1h2 += self.lr * numpy.dot((hidden2_errors * hidden2_outputs * (1.0 - hidden2_outputs)), numpy.transpose(hidden1_outputs))
        
        # update the weights for the links between the input and hidden layers
        self.wih1 += self.lr * numpy.dot((hidden1_errors * hidden1_outputs * (1.0 - hidden1_outputs)), numpy.transpose(inputs))
        
        pass

    
    # query the neural network
    def query(self, inputs_list):
        # convert inputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        
        # calculate signals into hidden layer
        hidden1_inputs = numpy.dot(self.wih1, inputs)
        # calculate the signals emerging from hidden layer
        hidden1_outputs = self.activation_function(hidden1_inputs)
        
        # calculate signals into hidden layer
        hidden2_inputs = numpy.dot(self.wh1h2, hidden1_outputs)
        # calculate the signals emerging from hidden layer
        hidden2_outputs = self.activation_function(hidden2_inputs)
        
        hidden3_inputs = numpy.dot(self.wh2h3, hidden2_outputs)
        hidden3_outputs = self.activation_function(hidden3_inputs)
        
        # calculate signals into final output layer
        final_inputs = numpy.dot(self.wh3o, hidden3_outputs)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs

In [4]:
# number of input, hidden and output nodes
input_nodes = 784
hidden_nodes1 = 500
hidden_nodes2 = 200
hidden_nodes3 = 100
output_nodes = 10

# learning rate
learning_rate = 0.01

# create instance of neural network
n2 = neuralNetwork_5Layers(input_nodes, hidden_nodes1, hidden_nodes2, hidden_nodes3, output_nodes, learning_rate)

epochs = 15

for e in range(epochs):
    # go through all records in the training data set
    for record in training_data_list:
        # split the record by the ',' commas
        all_values = record.split(',')
        # scale and shift the inputs
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        # create the target output values (all 0.01, except the desired label which is 0.99)
        targets = numpy.zeros(output_nodes) + 0.01
        # all_values[0] is the target label for this record
        targets[int(all_values[0])] = 0.99
        n2.train(inputs, targets)
        pass
    
    # scorecard for how well the network performs, initially empty
    scorecard = []

    # go through all the records in the test data set
    for record in test_data_list:
        # split the record by the ',' commas
        all_values = record.split(',')
        # correct answer is first value
        correct_label = int(all_values[0])
        # scale and shift the inputs
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        # query the network
        outputs = n2.query(inputs)
        # the index of the highest value corresponds to the label
        label = numpy.argmax(outputs)
        # append correct or incorrect to list
        if (label == correct_label):
            # network's answer matches correct answer, add 1 to scorecard
            scorecard.append(1)
        else:
            # network's answer doesn't match correct answer, add 0 to scorecard
            scorecard.append(0)
            pass

        pass

    # calculate the performance score, the fraction of correct answers
    scorecard_array = numpy.asarray(scorecard)
    print ("performance for ",1+e," epochs = ", scorecard_array.sum() / scorecard_array.size)
    pass

performance for  1  epochs =  0.9375
performance for  2  epochs =  0.9586
performance for  3  epochs =  0.9584
performance for  4  epochs =  0.9595
performance for  5  epochs =  0.9637
performance for  6  epochs =  0.9674
performance for  7  epochs =  0.9607
performance for  8  epochs =  0.9539
performance for  9  epochs =  0.9591
performance for  10  epochs =  0.9566
performance for  11  epochs =  0.9424
performance for  12  epochs =  0.9532
performance for  13  epochs =  0.96
performance for  14  epochs =  0.9551
performance for  15  epochs =  0.9582


### learning rate를 더 낮춰서 해봄

In [5]:
n2_1 = neuralNetwork_5Layers(input_nodes, hidden_nodes1, hidden_nodes2, hidden_nodes3, output_nodes, 0.001)

epochs = 15

for e in range(epochs):
    # go through all records in the training data set
    for record in training_data_list:
        # split the record by the ',' commas
        all_values = record.split(',')
        # scale and shift the inputs
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        # create the target output values (all 0.01, except the desired label which is 0.99)
        targets = numpy.zeros(output_nodes) + 0.01
        # all_values[0] is the target label for this record
        targets[int(all_values[0])] = 0.99
        n2_1.train(inputs, targets)
        pass
    
    # scorecard for how well the network performs, initially empty
    scorecard = []

    # go through all the records in the test data set
    for record in test_data_list:
        # split the record by the ',' commas
        all_values = record.split(',')
        # correct answer is first value
        correct_label = int(all_values[0])
        # scale and shift the inputs
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        # query the network
        outputs = n2_1.query(inputs)
        # the index of the highest value corresponds to the label
        label = numpy.argmax(outputs)
        # append correct or incorrect to list
        if (label == correct_label):
            # network's answer matches correct answer, add 1 to scorecard
            scorecard.append(1)
        else:
            # network's answer doesn't match correct answer, add 0 to scorecard
            scorecard.append(0)
            pass

        pass

    # calculate the performance score, the fraction of correct answers
    scorecard_array = numpy.asarray(scorecard)
    print ("performance for ",1+e," epochs = ", scorecard_array.sum() / scorecard_array.size)
    pass

performance for  1  epochs =  0.5431
performance for  2  epochs =  0.8367
performance for  3  epochs =  0.895
performance for  4  epochs =  0.9157
performance for  5  epochs =  0.9278
performance for  6  epochs =  0.9385
performance for  7  epochs =  0.946
performance for  8  epochs =  0.9526
performance for  9  epochs =  0.9571
performance for  10  epochs =  0.9603
performance for  11  epochs =  0.9625
performance for  12  epochs =  0.964
performance for  13  epochs =  0.9653
performance for  14  epochs =  0.9667
performance for  15  epochs =  0.9677


In [6]:
epochs = 15

for e in range(epochs):
    # go through all records in the training data set
    for record in training_data_list:
        # split the record by the ',' commas
        all_values = record.split(',')
        # scale and shift the inputs
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        # create the target output values (all 0.01, except the desired label which is 0.99)
        targets = numpy.zeros(output_nodes) + 0.01
        # all_values[0] is the target label for this record
        targets[int(all_values[0])] = 0.99
        n2_1.train(inputs, targets)
        pass
    
    # scorecard for how well the network performs, initially empty
    scorecard = []

    # go through all the records in the test data set
    for record in test_data_list:
        # split the record by the ',' commas
        all_values = record.split(',')
        # correct answer is first value
        correct_label = int(all_values[0])
        # scale and shift the inputs
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        # query the network
        outputs = n2_1.query(inputs)
        # the index of the highest value corresponds to the label
        label = numpy.argmax(outputs)
        # append correct or incorrect to list
        if (label == correct_label):
            # network's answer matches correct answer, add 1 to scorecard
            scorecard.append(1)
        else:
            # network's answer doesn't match correct answer, add 0 to scorecard
            scorecard.append(0)
            pass

        pass

    # calculate the performance score, the fraction of correct answers
    scorecard_array = numpy.asarray(scorecard)
    print ("performance for ",16+e," epochs = ", scorecard_array.sum() / scorecard_array.size)
    pass

performance for  16  epochs =  0.9679
performance for  17  epochs =  0.9689
performance for  18  epochs =  0.9696
performance for  19  epochs =  0.97
performance for  20  epochs =  0.972
performance for  21  epochs =  0.9714
performance for  22  epochs =  0.9715
performance for  23  epochs =  0.972
performance for  24  epochs =  0.9715
performance for  25  epochs =  0.9729
performance for  26  epochs =  0.9727
performance for  27  epochs =  0.9732
performance for  28  epochs =  0.9725
performance for  29  epochs =  0.9726
performance for  30  epochs =  0.9726


# 6Layer에서 구현

In [6]:
# neural network class definition
class neuralNetwork_6Layers:
    
    
     # initialise the neural network
    def __init__(self, inputnodes, hiddennodes1, hiddennodes2, hiddennodes3, hiddennodes4, outputnodes, learningrate):
        # set number of nodes in each input, hidden, output layer
        self.inodes = inputnodes
        self.hnodes1 = hiddennodes1
        self.hnodes2 = hiddennodes2
        self.hnodes3 = hiddennodes3
        self.hnodes4 = hiddennodes4
        self.onodes = outputnodes
        
        # link weight matrices, wih and who
        # weights inside the arrays are w_i_j, where link is from node i to node j in the next layer
        # w11 w21
        # w12 w22 etc 
        self.wih1 = numpy.random.normal(0.0, pow(self.inodes, -0.5), (self.hnodes1, self.inodes))
        self.wh1h2 = numpy.random.normal(0.0, pow(self.hnodes1, -0.5), (self.hnodes2, self.hnodes1))
        self.wh2h3 = numpy.random.normal(0.0, pow(self.hnodes2, -0.5), (self.hnodes3, self.hnodes2))
        self.wh3h4 = numpy.random.normal(0.0, pow(self.hnodes3, -0.5), (self.hnodes4, self.hnodes3))
        self.wh4o = numpy.random.normal(0.0, pow(self.hnodes4, -0.5), (self.onodes, self.hnodes4))

        # learning rate
        self.lr = learningrate
        
        # activation function is the sigmoid function
        self.activation_function = lambda x: scipy.special.expit(x)
        
        pass

    
    # train the neural network
    def train(self, inputs_list, targets_list):
        # convert inputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T
        
        # calculate signals into hidden layer
        hidden1_inputs = numpy.dot(self.wih1, inputs)
        # calculate the signals emerging from hidden layer
        hidden1_outputs = self.activation_function(hidden1_inputs)
        
        # calculate signals into hidden layer
        hidden2_inputs = numpy.dot(self.wh1h2, hidden1_outputs)
        # calculate the signals emerging from hidden layer
        hidden2_outputs = self.activation_function(hidden2_inputs)
        
        hidden3_inputs = numpy.dot(self.wh2h3, hidden2_outputs)
        hidden3_outputs = self.activation_function(hidden3_inputs)
        
        hidden4_inputs = numpy.dot(self.wh3h4, hidden3_outputs)
        hidden4_outputs = self.activation_function(hidden4_inputs)
        
        # calculate signals into final output layer
        final_inputs = numpy.dot(self.wh4o, hidden4_outputs)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
         # output layer error is the (target - actual)
        output_errors = targets - final_outputs
        # hidden layer error is the output_errors, split by weights, recombined at hidden nodes
        hidden4_errors = numpy.dot(self.wh4o.T, output_errors)
        hidden3_errors = numpy.dot(self.wh3h4.T, hidden4_errors)
        hidden2_errors = numpy.dot(self.wh2h3.T, hidden3_errors)
        hidden1_errors = numpy.dot(self.wh1h2.T, hidden2_errors)
        
        # update the weights for the links between the hidden and output layers
        self.wh4o += self.lr * numpy.dot((output_errors * final_outputs * (1.0 - final_outputs)), numpy.transpose(hidden4_outputs))
        self.wh3h4 += self.lr * numpy.dot((hidden4_errors * hidden4_outputs * (1.0 - hidden4_outputs)), numpy.transpose(hidden3_outputs))
        self.wh2h3 += self.lr * numpy.dot((hidden3_errors * hidden3_outputs * (1.0 - hidden3_outputs)), numpy.transpose(hidden2_outputs))
        self.wh1h2 += self.lr * numpy.dot((hidden2_errors * hidden2_outputs * (1.0 - hidden2_outputs)), numpy.transpose(hidden1_outputs))
        
        # update the weights for the links between the input and hidden layers
        self.wih1 += self.lr * numpy.dot((hidden1_errors * hidden1_outputs * (1.0 - hidden1_outputs)), numpy.transpose(inputs))
        
        pass

    
    # query the neural network
    def query(self, inputs_list):
        # convert inputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        
        # calculate signals into hidden layer
        hidden1_inputs = numpy.dot(self.wih1, inputs)
        # calculate the signals emerging from hidden layer
        hidden1_outputs = self.activation_function(hidden1_inputs)
        
        # calculate signals into hidden layer
        hidden2_inputs = numpy.dot(self.wh1h2, hidden1_outputs)
        # calculate the signals emerging from hidden layer
        hidden2_outputs = self.activation_function(hidden2_inputs)
        
        hidden3_inputs = numpy.dot(self.wh2h3, hidden2_outputs)
        hidden3_outputs = self.activation_function(hidden3_inputs)
        
        hidden4_inputs = numpy.dot(self.wh3h4, hidden3_outputs)
        hidden4_outputs = self.activation_function(hidden4_inputs)
        
        # calculate signals into final output layer
        final_inputs = numpy.dot(self.wh4o, hidden4_outputs)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs

In [10]:
# number of input, hidden and output nodes
input_nodes = 784
hidden_nodes1 = 500
hidden_nodes2 = 300
hidden_nodes3 = 200
hidden_nodes4 = 100
output_nodes = 10

# learning rate
learning_rate = 0.01

# create instance of neural network
n3 = neuralNetwork_6Layers(input_nodes, hidden_nodes1, hidden_nodes2, hidden_nodes3, hidden_nodes4, output_nodes, learning_rate)

epochs = 15

for e in range(epochs):
    # go through all records in the training data set
    for record in training_data_list:
        # split the record by the ',' commas
        all_values = record.split(',')
        # scale and shift the inputs
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        # create the target output values (all 0.01, except the desired label which is 0.99)
        targets = numpy.zeros(output_nodes) + 0.01
        # all_values[0] is the target label for this record
        targets[int(all_values[0])] = 0.99
        n3.train(inputs, targets)
        pass
    
    # scorecard for how well the network performs, initially empty
    scorecard = []

    # go through all the records in the test data set
    for record in test_data_list:
        # split the record by the ',' commas
        all_values = record.split(',')
        # correct answer is first value
        correct_label = int(all_values[0])
        # scale and shift the inputs
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        # query the network
        outputs = n3.query(inputs)
        # the index of the highest value corresponds to the label
        label = numpy.argmax(outputs)
        # append correct or incorrect to list
        if (label == correct_label):
            # network's answer matches correct answer, add 1 to scorecard
            scorecard.append(1)
        else:
            # network's answer doesn't match correct answer, add 0 to scorecard
            scorecard.append(0)
            pass

        pass

    # calculate the performance score, the fraction of correct answers
    scorecard_array = numpy.asarray(scorecard)
    print ("performance for ",1+e," epochs = ", scorecard_array.sum() / scorecard_array.size)
    pass

performance for  1  epochs =  0.898
performance for  2  epochs =  0.7796
performance for  3  epochs =  0.7982
performance for  4  epochs =  0.761
performance for  5  epochs =  0.5523
performance for  6  epochs =  0.6507
performance for  7  epochs =  0.6885
performance for  8  epochs =  0.7175
performance for  9  epochs =  0.4916
performance for  10  epochs =  0.408
performance for  11  epochs =  0.5318
performance for  12  epochs =  0.3869
performance for  13  epochs =  0.4583
performance for  14  epochs =  0.5777
performance for  15  epochs =  0.5017


In [11]:
# create instance of neural network
n3_1 = neuralNetwork_6Layers(input_nodes, hidden_nodes1, hidden_nodes2, hidden_nodes3, hidden_nodes4, output_nodes, 0.001)

epochs = 15

for e in range(epochs):
    # go through all records in the training data set
    for record in training_data_list:
        # split the record by the ',' commas
        all_values = record.split(',')
        # scale and shift the inputs
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        # create the target output values (all 0.01, except the desired label which is 0.99)
        targets = numpy.zeros(output_nodes) + 0.01
        # all_values[0] is the target label for this record
        targets[int(all_values[0])] = 0.99
        n3_1.train(inputs, targets)
        pass
    
    # scorecard for how well the network performs, initially empty
    scorecard = []

    # go through all the records in the test data set
    for record in test_data_list:
        # split the record by the ',' commas
        all_values = record.split(',')
        # correct answer is first value
        correct_label = int(all_values[0])
        # scale and shift the inputs
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        # query the network
        outputs = n3_1.query(inputs)
        # the index of the highest value corresponds to the label
        label = numpy.argmax(outputs)
        # append correct or incorrect to list
        if (label == correct_label):
            # network's answer matches correct answer, add 1 to scorecard
            scorecard.append(1)
        else:
            # network's answer doesn't match correct answer, add 0 to scorecard
            scorecard.append(0)
            pass

        pass

    # calculate the performance score, the fraction of correct answers
    scorecard_array = numpy.asarray(scorecard)
    print ("performance for ",1+e," epochs = ", scorecard_array.sum() / scorecard_array.size)
    pass

performance for  1  epochs =  0.517
performance for  2  epochs =  0.7548
performance for  3  epochs =  0.8874
performance for  4  epochs =  0.9226
performance for  5  epochs =  0.9409
performance for  6  epochs =  0.9526
performance for  7  epochs =  0.9592
performance for  8  epochs =  0.9637
performance for  9  epochs =  0.9684
performance for  10  epochs =  0.9703
performance for  11  epochs =  0.97
performance for  12  epochs =  0.9704
performance for  13  epochs =  0.9708
performance for  14  epochs =  0.9703
performance for  15  epochs =  0.9707


In [12]:
epochs = 15

for e in range(epochs):
    # go through all records in the training data set
    for record in training_data_list:
        # split the record by the ',' commas
        all_values = record.split(',')
        # scale and shift the inputs
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        # create the target output values (all 0.01, except the desired label which is 0.99)
        targets = numpy.zeros(output_nodes) + 0.01
        # all_values[0] is the target label for this record
        targets[int(all_values[0])] = 0.99
        n3_1.train(inputs, targets)
        pass
    
    # scorecard for how well the network performs, initially empty
    scorecard = []

    # go through all the records in the test data set
    for record in test_data_list:
        # split the record by the ',' commas
        all_values = record.split(',')
        # correct answer is first value
        correct_label = int(all_values[0])
        # scale and shift the inputs
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        # query the network
        outputs = n3_1.query(inputs)
        # the index of the highest value corresponds to the label
        label = numpy.argmax(outputs)
        # append correct or incorrect to list
        if (label == correct_label):
            # network's answer matches correct answer, add 1 to scorecard
            scorecard.append(1)
        else:
            # network's answer doesn't match correct answer, add 0 to scorecard
            scorecard.append(0)
            pass

        pass

    # calculate the performance score, the fraction of correct answers
    scorecard_array = numpy.asarray(scorecard)
    print ("performance for ",16+e," epochs = ", scorecard_array.sum() / scorecard_array.size)
    pass

performance for  16  epochs =  0.9711
performance for  17  epochs =  0.9701
performance for  18  epochs =  0.9716
performance for  19  epochs =  0.9733
performance for  20  epochs =  0.9731
performance for  21  epochs =  0.971
performance for  22  epochs =  0.9697
performance for  23  epochs =  0.9704
performance for  24  epochs =  0.9709
performance for  25  epochs =  0.9709
performance for  26  epochs =  0.9714
performance for  27  epochs =  0.9705
performance for  28  epochs =  0.9664
performance for  29  epochs =  0.9673
performance for  30  epochs =  0.9688


# 각각 조건에 따라 1 epoch 도는데 걸리는 시간 측정

### 3layer

In [7]:
import time

In [15]:
# number of input, hidden and output nodes
input_nodes = 784
hidden_nodes = 200
output_nodes = 10

# learning rate
learning_rate = 0.1

# create instance of neural network
n = neuralNetwork(input_nodes,hidden_nodes,output_nodes, learning_rate)

epochs = 1

start_time = time.time()

for e in range(epochs):
    # go through all records in the training data set
    for record in training_data_list:
        # split the record by the ',' commas
        all_values = record.split(',')
        # scale and shift the inputs
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        # create the target output values (all 0.01, except the desired label which is 0.99)
        targets = numpy.zeros(output_nodes) + 0.01
        # all_values[0] is the target label for this record
        targets[int(all_values[0])] = 0.99
        n.train(inputs, targets)
        pass

total_time = time.time() - start_time
print("3Layer lr=0.1 learning time per epoch = ", total_time)

n_1 = neuralNetwork(input_nodes,hidden_nodes,output_nodes, 0.01)

epochs = 1

start_time_1 = time.time()

for e in range(epochs):
    # go through all records in the training data set
    for record in training_data_list:
        # split the record by the ',' commas
        all_values = record.split(',')
        # scale and shift the inputs
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        # create the target output values (all 0.01, except the desired label which is 0.99)
        targets = numpy.zeros(output_nodes) + 0.01
        # all_values[0] is the target label for this record
        targets[int(all_values[0])] = 0.99
        n_1.train(inputs, targets)
        pass

total_time_1 = time.time() - start_time_1
print("3Layer lr=0.01 learning time per epoch = ", total_time_1)

3Layer lr=0.1 learning time per epoch =  80.38451933860779
3Layer lr=0.01 learning time per epoch =  76.57321619987488


### 4Layer

##### CPU 57%정도씩 점유하면서 계산함

In [12]:
# number of input, hidden and output nodes
input_nodes = 784
hidden_nodes1 = 500
hidden_nodes2 = 200
output_nodes = 10

# learning rate
learning_rate = 0.1

# create instance of neural network
n1 = neuralNetwork_4Layers(input_nodes,hidden_nodes1,hidden_nodes2, output_nodes, learning_rate)

epochs = 1

start_time = time.time()

for e in range(epochs):
    # go through all records in the training data set
    for record in training_data_list:
        # split the record by the ',' commas
        all_values = record.split(',')
        # scale and shift the inputs
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        # create the target output values (all 0.01, except the desired label which is 0.99)
        targets = numpy.zeros(output_nodes) + 0.01
        # all_values[0] is the target label for this record
        targets[int(all_values[0])] = 0.99
        n1.train(inputs, targets)
        pass

total_time = time.time() - start_time
print("4Layer lr=0.1 learning time per epoch = ", total_time)

n1_1 = neuralNetwork_4Layers(input_nodes,hidden_nodes1,hidden_nodes2, output_nodes, 0.01)

start_time_1 = time.time()

for e in range(epochs):
    # go through all records in the training data set
    for record in training_data_list:
        # split the record by the ',' commas
        all_values = record.split(',')
        # scale and shift the inputs
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        # create the target output values (all 0.01, except the desired label which is 0.99)
        targets = numpy.zeros(output_nodes) + 0.01
        # all_values[0] is the target label for this record
        targets[int(all_values[0])] = 0.99
        n1_1.train(inputs, targets)
        pass

total_time_1 = time.time() - start_time_1
print("4Layer lr=0.01 learning time per epoch = ", total_time_1)

4Layer lr=0.1 learning time per epoch =  258.96376514434814
4Layer lr=0.01 learning time per epoch =  233.25036525726318


### 5Layer

##### CPU 57%정도씩  점유하면서 계산

In [11]:
# number of input, hidden and output nodes
input_nodes = 784
hidden_nodes1 = 500
hidden_nodes2 = 200
hidden_nodes3 = 100
output_nodes = 10

# learning rate
learning_rate = 0.01

# create instance of neural network
n2 = neuralNetwork_5Layers(input_nodes, hidden_nodes1, hidden_nodes2, hidden_nodes3, output_nodes, learning_rate)

epochs = 1

start_time = time.time()

for e in range(epochs):
    # go through all records in the training data set
    for record in training_data_list:
        # split the record by the ',' commas
        all_values = record.split(',')
        # scale and shift the inputs
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        # create the target output values (all 0.01, except the desired label which is 0.99)
        targets = numpy.zeros(output_nodes) + 0.01
        # all_values[0] is the target label for this record
        targets[int(all_values[0])] = 0.99
        n2.train(inputs, targets)
        pass

total_time = time.time() - start_time
print("5Layer lr=0.01 learning time per epoch = ", total_time)

n2_1 = neuralNetwork_5Layers(input_nodes, hidden_nodes1, hidden_nodes2, hidden_nodes3, output_nodes, 0.001)


start_time_1 = time.time()

for e in range(epochs):
    # go through all records in the training data set
    for record in training_data_list:
        # split the record by the ',' commas
        all_values = record.split(',')
        # scale and shift the inputs
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        # create the target output values (all 0.01, except the desired label which is 0.99)
        targets = numpy.zeros(output_nodes) + 0.01
        # all_values[0] is the target label for this record
        targets[int(all_values[0])] = 0.99
        n2_1.train(inputs, targets)
        pass

total_time_1 = time.time() - start_time_1
print("5Layer lr=0.001 learning time per epoch = ", total_time_1)

5Layer lr=0.01 learning time per epoch =  265.59662103652954
5Layer lr=0.001 learning time per epoch =  240.45472264289856


### 6Layer

##### 이것도 CPU 57프로 정도 점유

In [13]:
# number of input, hidden and output nodes
input_nodes = 784
hidden_nodes1 = 500
hidden_nodes2 = 300
hidden_nodes3 = 200
hidden_nodes4 = 100
output_nodes = 10

# learning rate
learning_rate = 0.01

# create instance of neural network
n3 = neuralNetwork_6Layers(input_nodes, hidden_nodes1, hidden_nodes2, hidden_nodes3, hidden_nodes4, output_nodes, learning_rate)

epochs = 1

start_time = time.time()

for e in range(epochs):
    # go through all records in the training data set
    for record in training_data_list:
        # split the record by the ',' commas
        all_values = record.split(',')
        # scale and shift the inputs
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        # create the target output values (all 0.01, except the desired label which is 0.99)
        targets = numpy.zeros(output_nodes) + 0.01
        # all_values[0] is the target label for this record
        targets[int(all_values[0])] = 0.99
        n3.train(inputs, targets)
        pass

total_time = time.time() - start_time
print("6Layer lr=0.01 learning time per epoch = ", total_time)

n3_1 = neuralNetwork_6Layers(input_nodes, hidden_nodes1, hidden_nodes2, hidden_nodes3, hidden_nodes4, output_nodes, 0.001)

start_time_1 = time.time()

for e in range(epochs):
    # go through all records in the training data set
    for record in training_data_list:
        # split the record by the ',' commas
        all_values = record.split(',')
        # scale and shift the inputs
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        # create the target output values (all 0.01, except the desired label which is 0.99)
        targets = numpy.zeros(output_nodes) + 0.01
        # all_values[0] is the target label for this record
        targets[int(all_values[0])] = 0.99
        n3_1.train(inputs, targets)
        pass

total_time_1 = time.time() - start_time_1
print("6Layer lr=0.01 learning time per epoch = ", total_time_1)

6Layer lr=0.01 learning time per epoch =  317.9898991584778
6Layer lr=0.01 learning time per epoch =  303.97754979133606


In [14]:
## 앞에 출력문에 오타가 있었내요;;
print("6Layer lr=0.001 learning time per epoch = ", total_time_1)

6Layer lr=0.001 learning time per epoch =  303.97754979133606
